In [1]:
import cv2
import numpy as np

In [2]:
corner_track_params = dict(maxCorners = 10,
                          qualityLevel = 0.3,
                          minDistance = 7,
                          blockSize = 7)

#SHi tomasi corner detection

In [3]:
# Detect the motion of specific points or the aggregated motion of regions by modifying the winSize argument. This determines the integration window size. Small windows are more sensitive to noise and may miss larger motions. Large windows will “survive” an occlusion.

# The integration appears smoother with the larger window size.

# criteria has two here - the max number (10 above) of iterations and epsilon (0.03 above). More iterations means a more exhaustive search, and a smaller epsilon finishes earlier. These are primarily useful in exchanging speed vs accuracy, but mainly stay the same.

# When maxLevel is 0, it is the same algorithm without using pyramids (ie, calcOpticalFlowLK). Pyramids allow finding optical flow at various resolutions of the image. 

In [4]:
#lukas kanade method params

lk_params = dict(winSize = (200, 200),
                maxLevel = 2,
                criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

In [6]:
cap = cv2.VideoCapture(0)

#take the first video frame
ret, prev_frame = cap.read()

prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

prevPts = cv2.goodFeaturesToTrack(prev_gray, mask = None, **corner_track_params)

#make a mask of the prev frame size to draw later
mask = np.zeros_like(prev_frame)

while True:
    
    ret, frame = cap.read()
    
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    #optical flow on the grayscale img
    nextPts, status, err = cv2.calcOpticalFlowPyrLK(prev_gray, frame_gray, prevPts, None, **lk_params)
    
    # Using the returned status array (the status output)
    # status output status vector (of unsigned chars); each element of the vector is set to 1 if
    # the flow for the corresponding features has been found, otherwise, it is set to 0
    
    good_new = nextPts[status == 1]
    good_prev = prevPts[status == 1]
    
    for i, (new, prev) in enumerate(zip(good_new, good_prev)):
        
        x_new, y_new = new.ravel()  #to draw on later we get each of the points separately
        x_prev, y_prev = prev.ravel()
        
        mask = cv2.line(mask, (x_new, y_new), (x_prev, y_prev), (0, 255, 0), 3)
        
        frame = cv2.circle(frame, (x_new, y_new), 8, (0, 0, 255), -1)
        
    img = cv2.add(frame, mask)
    cv2.imshow("Object Flow", img)
    
    k = cv2.waitKey(30) & 0xFF
    if k == 27:
        break
        
    #update the prev frame and the pts with the new one for the next iteration
    
    prev_gray = frame_gray.copy()
    prevPts = good_new.reshape(-1, 1, 2)
    
cv2.destroyAllWindows()
cap.release()